# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [7]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [8]:
from typing import List, Tuple
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin
from azure.ai.projects.models import MessageRole, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
BING_GROUNDING_PROJECT_CONNECTION_STRING = os.getenv("BING_GROUNDING_PROJECT_CONNECTION_STRING")
BING_GROUNDING_AGENT_ID = os.getenv("BING_GROUNDING_AGENT_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
# Web search mode: "google" or "bing"
# it can be changed when users want to use different search engine
WEB_SEARCH_MODE = os.getenv("WEB_SEARCH_MODE")
WEB_CRAWLING_MODE = os.getenv("WEB_CRAWLING_MODE") #on or off

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text



async def extract_contexts_async(url_snippet_tuples: List[Tuple[str, str]]) -> List[str]:
    """
    Asynchronously extract content from a list of URLs with their snippets.
    
    Args:
        url_snippet_tuples: List of (url, snippet) pairs to process
        
    Returns:
        List of extracted contents
    """
    async def fetch(url: str, snippet: str) -> str:
        # Try to get from cache first
        
        # If not in cache or cache unavailable, fetch the content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        
        try:
            async with httpx.AsyncClient(timeout=10, follow_redirects=True) as client:
                try:
                    response = await client.get(url, headers=headers)
                    response.raise_for_status()
                except httpx.HTTPStatusError as e:
                    # Handle redirects manually if needed
                    if e.response.status_code == 302 and "location" in e.response.headers:
                        redirect_url = e.response.headers["location"]
                        if not redirect_url.startswith("http"):
                            redirect_url = urljoin(url, redirect_url)
                        try:
                            response = await client.get(redirect_url, headers=headers)
                            response.raise_for_status()
                        except Exception as e2:
                            print(f"Redirect request failed: {e2}")
                            return f"{snippet} "
                    else:
                        print(f"Request failed: {e}")
                        return f"{snippet} "
                except httpx.HTTPError as e:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
                
                # Parse the content
                selector = scrapy.Selector(text=response.text)
                
                # Extract paragraphs
                paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
                
                # Remove duplicate and very short paragraphs
                filtered_paragraphs = []
                seen_content = set()
                for p in paragraphs:
                    # Skip very short paragraphs that are likely UI elements
                    if len(p) < 5:
                        continue
                    # Avoid duplicate content
                    if p in seen_content:
                        continue
                    seen_content.add(p)
                    filtered_paragraphs.append(p)
                
                # Join the filtered paragraphs
                text = "\n".join(filtered_paragraphs)
                
                # If no paragraphs were found, try to get other text content
                if not text:
                    content_texts = [t.strip() for t in selector.css(
                        'article::text, article *::text, .content::text, .content *::text, '
                        'main::text, main *::text'
                    ).getall() if t.strip()]
                    
                    if content_texts:
                        text = "\n".join(content_texts)
                
                # Combine snippet with extracted text
                snippet_text = f"{snippet}: {text}"
                
                
                return snippet_text
                
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            return f"{snippet} [Error: {str(e)}]"
    
    # Create tasks for all URLs
    tasks = [asyncio.create_task(fetch(url, snippet)) 
            for url, snippet in url_snippet_tuples]
    
    # Execute all tasks concurrently
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Process results
    processed_results = []
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Error processing URL {url_snippet_tuples[i][0]}: {str(result)}")
            processed_results.append(f"{url_snippet_tuples[i][1]} [Processing Error]")
        else:
            processed_results.append({"content": result, "url_citation" :{"link": url_snippet_tuples[i][0], "title": url_snippet_tuples[i][1]}})

    return processed_results
                    
           
def google_search(query, num=5, search_type="web"):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e"
    }
    
    if search_type == "image":
        params["searchType"] = "image"
        
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

def bing_grounding_search(query, num=5, search_type="web"):
    try:
        creds = DefaultAzureCredential()
        
        project_client = AIProjectClient.from_connection_string(
            credential=creds,
            conn_str=BING_GROUNDING_PROJECT_CONNECTION_STRING,
        )
        
        agent_id = BING_GROUNDING_AGENT_ID
        
        if not agent_id:
            print("BING_GROUNDING_AGENT_ID is not set. Create new agent...")
            connection_name = BING_GROUNDING_CONNECTION_NAME
            
            bing_connection = project_client.connections.get(
                connection_name=connection_name,
            )
            conn_id = bing_connection.id
            
            bing = BingGroundingTool(connection_id=conn_id)
            
            
            agent = project_client.agents.create_agent(
                model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,
                name="temporary-bing-agent",
                instructions="""
                    Search for product information and images exclusively about Samsung products. Get all contents from the website as much as you can. Don't include the url link in the response.
                    Only respond with information from trusted sources: samsung.com.
                    Prioritize data from samsung.com whenever available to ensure accuracy and reliability.
                    If information is not found on samsung.com, supplement with tistory.com, but always indicate the source.
                    Avoid using data from any other websites or unverified sources.
                """,
                tools=bing.definitions,
                headers={"x-ms-enable-preview": "true"}
            )
            agent_id = agent.id
            print(f"New agent created. Agent ID: {agent_id}")
        else:
            print(f"Existing agent ID: {agent_id}")
            try:
                agent = project_client.agents.get_agent(agent_id)
            except Exception as agent_error:
                print(f"Failed to retrieve agent: {agent_error}")
                return []

        thread = project_client.agents.create_thread()
        
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"Search the web for: {query}. Return only the top {num} most relevant results as a list.",
        )

        
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        
        if run.status == "failed":
            print(f"Execution failed: {run.last_error}")
            return []
        
        results = []
        response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
            MessageRole.AGENT
        )
        # Extract content text and annotations
        if response_message.content:
            for content_item in response_message["content"]:
                if content_item["type"] == "text":
                    text_content = content_item["text"]["value"]
                    results.append({"content": text_content})
            
        if response_message.url_citation_annotations:
            for annotation in response_message.url_citation_annotations:
                if annotation["type"] == "url_citation":
                    url_citation = annotation["url_citation"]
                    url = url_citation["url"]
                    title = url_citation["title"]
                    results.append({"url_citation":{"link": url, "title": title}})

        if not BING_GROUNDING_AGENT_ID and hasattr(agent, 'id'):
            try:
                print(f"Deleting temporary agent with ID: {agent.id}")
                project_client.agents.delete_agent(agent.id)
                
            except Exception as delete_error:
                print(f"Error deleting agent: {delete_error}")

        return results if results else []
    except Exception as e:
        print(f"Bing Grounding error : {e}")
        return []

def web_search(query, num=5, search_type="web"):
    """환경 변수에 따라 Google Search API 또는 Bing Grounding을 사용하여 검색 수행"""
    
    if WEB_SEARCH_MODE == "bing":
        try:
            return bing_grounding_search(query['llm_query'], num, search_type)
            
        except Exception as e:
            print(f"Bing Grounding 검색 중 오류 발생: {e}")
    else:
        return google_search(query['web_search'], num, search_type)

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Web Search용 Query Rewrite:
            - 사용자의 질문을 실제 검색 엔진 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 웹 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"web_search": "웹 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())


In [9]:
from IPython.display import Markdown, display
from datetime import datetime
import time

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

async def process_web_search_call(RESULTS_COUNT, input):
    
    start_time = time.time()
    
    contexts = [] 
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Web Search Query: {query_rewrite['web_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = web_search(query_rewrite, RESULTS_COUNT)
    print(f"Web Search Results: {len(results)}")
    if WEB_SEARCH_MODE == "bing" and results and isinstance(results, list) and len(results) > 0:
        contexts = [results[i] for i in range(len(results))]
        
    elif WEB_SEARCH_MODE == "google" and results and isinstance(results, list) and len(results) > 0:
        url_snippet_tuples = [(r["link"], r["snippet"]) for r in results]
        contexts = await extract_contexts_async(url_snippet_tuples)
        
    else:
        print("No results found or invalid response from web_search.")
        contexts = [] 
    
    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = """
        너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 
        답변은 마크다운으로 이모지를 1~2개 포함해서 작성해줘. 
        contexts를 최대한 활용하여 풍부하게 답변을 해야해. 
        사용자가 질문한 내용에 대해 정확하고 유용한 정보를 제공해야 해. contexts가 부족하면 최소한의 안내만 해줘. 
        url_citation은 사용자가 클릭할 수 있도록 링크를 제공해줘.
        
    """
    user_prompt = f"""
        너는 아래 제공하는 웹검색에서 검색한 contexts를 바탕으로 질문에 대한 답변을 제공해야 해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.
        웹검색에서 제공한 contexts: {contexts}
        질문: {query_rewrite['llm_query']}
        """
    
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds")


In [10]:

RESULTS_COUNT = 3

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


WEB_SEARCH_MODE = "google"  

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)

WEB_SEARCH_MODE = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)    

Google Search API 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌 다른 인덕션 제품을 추천해 주세요. 3구 이상 모델이나 다양한 화구 수를 가진 인덕션의 주요 기능과 장점을 알려주시면 감사하겠습니다.
Web Search Results: 3


삼성전자의 다양한 인덕션 제품 중 3구 이상 모델을 고려할 때, 여러 기능과 장점이 있습니다. 특히, 최신 인덕션 제품은 다음과 같은 주요 기능을 갖추고 있습니다:

1. **고강도 상판**: 내구성이 뛰어나고 세련된 디자인으로 주방의 미관을 한층 높여줍니다. 💎

2. **진동감지센서**: 물 끓임 방지 기능이 있어 안전하게 요리를 할 수 있습니다. 이 센서는 물이 끓어 넘치는 것을 감지하여 자동으로 전원을 차단합니다.

3. **스마트 쿡**: 삼성의 스마트싱스 앱과 연동되어 요리를 보다 쉽게 할 수 있도록 도와줍니다. 요리 레시피를 참고하거나, 각 화구의 상태를 실시간으로 확인할 수 있습니다.

4. **고출력 화구**: 큰 화구의 출력이 2,700W 이상인 경우, 가스레인지 이상의 강력한 화력을 제공합니다. 🔥 이를 통해 빠르고 효율적인 요리가 가능합니다.

5. **설치방식**: 원스탑 설치 방식으로, 설치가 간편하여 주방에 쉽게 통합할 수 있습니다.

이러한 기능을 통해 삼성전자의 인덕션 제품은 보다 편리하고 안전한 요리를 제공하며, 다양한 화구 수를 통해 다채로운 요리를 동시에 할 수 있는 장점이 있습니다.

더 자세한 정보를 원하시면 [여기](https://blog.naver.com/yookura/223687436934)를 클릭하여 확인해 보세요!

elapsed time: 4.88 seconds
Bing Grounding 검색 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 제품 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 설명해 주시면 좋겠습니다.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 4


삼성전자의 3구 이상 인덕션 제품 중에서 추천할 만한 모델은 다음과 같습니다. 각 모델의 주요 기능과 장점도 함께 안내해 드릴게요! 🍳✨

### 1. BESPOKE AI Induction Infinite Line (NZ63DB857CAV)
- **주요 기능**: 
  - 세 개의 화구에서 동시에 사용 가능
  - 자동 전력 조절 기능
  - MAX Boost 기능으로 고출력 요리 가능 (최대 8분간)
  - 세련된 디자인
- **장점**: 
  - 높은 효율의 요리 성능
  - 화구 사용의 유연성
  - 프리미엄 디자인으로 주방 인테리어에 잘 어울림
- [자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz63db857cav-d2c/NZ63DB857CAV/)

### 2. Induction Cooktop (NZ63B5056AK)
- **주요 기능**:
  - 대형 이중 링 화구
  - Quattro Flex Zone으로 다채로운 요리 가능
  - 최대 출력 3,400W
- **장점**: 
  - 큰 조리기를 수용할 수 있는 넉넉한 공간
  - 강력한 난방 성능
  - 유연한 요리 구역으로 다양한 요리 스타일 지원
- [자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz63b5056cb-d2c/NZ63B5056AK/)

### 3. BESPOKE Induction Infinite Line (All Flex Zone, NZ64B9899RA)
- **주요 기능**:
  - 8개의 코일로 균일한 열 분배
  - 경계 없는 요리 구역
  - 모든 화구를 동시에 사용할 수 있으며, 총 출력 7,400W
- **장점**: 
  - 대규모 요리에 최적화
  - 균일한 조리로 맛을 극대화
  - 고급 기능을 통해 요리의 편리함 증가
- [자세히 보기](https://www.samsung.com/sec/ai-subs-kitchen/NZ64B9899RA-smart/NZ64B9899RA/)

이 모델들은 모두 다양한 요리를 효율적으로 지원하며, 주방의 스타일을 한층 더 높여줄 수 있습니다! 😊

elapsed time: 17.86 seconds


In [11]:

RESULTS_COUNT = 3

inputs = [
    #"삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


WEB_SEARCH_MODE = "google"  

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)

WEB_SEARCH_MODE = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)    

Google Search API 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


Web Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물할만한 삼성전자 TV를 추천해주세요. 추천 이유와 각 모델의 주요 기능도 함께 설명해 주시면 좋겠습니다.
Web Search Results: 3


부모님께 선물할 삼성전자 TV로는 **UN49MU6250F**와 **UN32K4110BF** 모델을 추천드립니다. 두 모델 모두 훌륭한 화질과 신뢰할 수 있는 A/S를 제공합니다. 😊

### 1. 삼성전자 UN49MU6250F
- **해상도**: 4K UHD
- **가격**: 약 68만원대
- **주요 기능**:
  - **고해상도**: 4K 해상도로 선명한 화질을 제공하여 영화나 드라마 감상 시 훨씬 더 몰입할 수 있습니다.
  - **스마트 기능**: 다양한 스트리밍 서비스와 앱을 지원하여 사용이 편리합니다.
  - **디자인**: 슬림한 디자인으로 집안 인테리어와 잘 어울립니다.
  
[자세히 보기](https://blog.naver.com/danawaelec/221259552788?viewType=pc)

---

### 2. 삼성전자 시리즈4 UN32K4110BF
- **해상도**: HD
- **가격**: 약 25만원대
- **주요 기능**:
  - **소형화**: 32형 화면으로 공간이 좁은 집에서도 적합합니다.
  - **기본적인 스마트 기능**: TV를 통해 다양한 콘텐츠를 쉽게 시청할 수 있습니다.
  - **가성비**: 적당한 가격대에 필요한 기능을 갖추고 있어 효도 선물로 적합합니다.

[자세히 보기](https://blog.naver.com/danawaelec/220909504564?viewType=pc)

이 두 모델은 각각의 용도와 예산에 맞춰 선택할 수 있어, 부모님께서 만족하실 만한 TV가 될 것입니다. 🎁

elapsed time: 5.99 seconds
Google Search API 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 2025년 제품 작년 대비 개선점
LLM Query: 삼성전자의 2025년 제품이 작년 모델에 비해 어떤 점에서 개선되었는지 구체적으로 설명해 주세요. 특히 주요 변화나 기능 향상에 대해 알고 싶습니다.
Web Search Results: 3


삼성전자는 2025년에 몇 가지 주요 제품 개선을 계획하고 있습니다. 특히 주목할 만한 변화는 고대역폭메모리(HBM)와 파운드리 기술에서 나타납니다. 🌟

1. **고대역폭메모리(HBM3E)**: 삼성전자는 5세대 HBM인 HBM3E의 개선 제품을 출시할 예정입니다. 이는 작년 HBM 공급의 경쟁력을 강화하고 SK하이닉스와의 경쟁에서 우위를 점하기 위한 전략으로 보입니다. 개선된 제품은 데이터 전송 속도 및 전력 효율성에서의 향상을 포함할 것으로 예상됩니다. [자세히 보기](https://blog.naver.com/irasu2/223744428165)

2. **파운드리 기술**: 삼성전자는 2025년에는 2나노미터 공정 생산을 목표로 하고 있습니다. 이 기술은 반도체 성능을 크게 향상시키고, 소형화와 전력 효율성을 높이는 데 기여할 것입니다. 이는 앞으로의 스마트폰, AI 및 IoT 디바이스에 중요한 역할을 할 것으로 예상됩니다. [자세히 보기](https://blog.naver.com/kntimes22/223818657091)

이러한 개선 사항들은 삼성전자가 글로벌 반도체 시장에서 경쟁력을 유지하고 혁신을 지속하는 데 중요한 요소가 될 것입니다. 🚀

elapsed time: 5.84 seconds
Bing Grounding 검색 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Web Search Query: 삼성전자 TV 추천 부모님 선물용
LLM Query: 부모님에게 선물할 삼성전자 TV를 추천해 주세요. 사용 용도나 추천할 만한 모델의 특징과 장점을 함께 설명해 주시면 좋겠습니다.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 1


부모님께 선물할 삼성전자 TV를 고려하신다면, 몇 가지 추천 모델과 그 특징을 알려드릴게요. 📺✨

### 1. 삼성 QLED TV
- **특징**: 삼성 QLED TV는 뛰어난 화질과 색감으로 유명합니다. Quantum Dot 기술을 사용해 생생하고 깊이 있는 색상을 표현할 수 있습니다.
- **장점**: 넓은 시야각과 높은 밝기로 다양한 조명 환경에서도 최적의 시청 경험을 제공합니다. 특히 스포츠나 액션 영화 같은 빠른 장면에서도 잔상 없이 부드럽게 시청할 수 있습니다.

### 2. 삼성 Neo QLED TV
- **특징**: Neo QLED는 미니 LED 기술을 사용하여 더욱 정밀한 밝기 조절이 가능하며, 블랙 표현력이 뛰어납니다.
- **장점**: HDR(High Dynamic Range) 기술로 다양한 명암비를 지원하여, 어두운 장면에서도 세밀한 디테일을 놓치지 않아요. 또한, 게임 모드와 스마트 기능이 강화되어 있어, 게임이나 스트리밍을 즐기기에 매우 적합합니다.

### 3. 삼성 Smart TV
- **특징**: 스마트 기능이 탑재된 TV로, 다양한 앱과 스트리밍 서비스를 쉽게 이용할 수 있습니다.
- **장점**: 가족이 함께 사용할 수 있는 다양한 콘텐츠에 접근할 수 있으며, 스마트 홈 기기와의 연동도 가능해 편리함을 더합니다. 음성 인식 기능이 있어 조작도 간편합니다.

### 선택 팁
부모님의 사용 용도에 따라 선택해보세요. 예를 들어, 영화 감상을 많이 하신다면 QLED나 Neo QLED가 좋고, 다양한 콘텐츠를 즐기실 계획이라면 Smart TV가 적합합니다.

더 자세한 정보는 [삼성 TV의 모든 것](https://www.samsung.com)에서 확인해 보실 수 있습니다. 부모님께서도 만족하실 멋진 선택이 되길 바랍니다! 🎁

elapsed time: 13.27 seconds
Bing Grounding 검색 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 25년 제품 작년 대비 개선 사항
LLM Query: 삼성전자에서 출시한 25년 제품이 작년 대비 어떤 점에서 개선되었는지 구체적으로 설명해 주세요. 제품의 기능, 성능, 디자인 등 다양한 측면에서의 변화에 대한 정보를 알고 싶습니다.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 4


2025년 삼성전자의 신제품은 여러 면에서 지난해와 비교하여 놀라운 개선을 이루었습니다. 아래에서 주요 변화 사항을 소개하겠습니다. 🌟

### 1. 디자인
- **경량 티타늄 프레임**: 삼성은 제품의 내구성을 높이기 위해 가벼운 티타늄 재질을 사용했습니다. 이는 사용자의 편리함을 더해주고, 디자인적으로도 세련된 느낌을 줍니다. 

### 2. 성능
- **향상된 프로세서**: 2025년 모델은 최신의 고성능 프로세서를 탑재하여 처리 속도와 반응성이 크게 개선되었습니다. 이는 특히 게임이나 멀티태스킹 시에 더욱 부드러운 경험을 제공합니다. 

### 3. AI 기능
- **제미니 AI 통합**: 새로운 AI 기능이 도입되어 사용자의 필요에 맞춰 최적화된 경험을 제공합니다. 예를 들어, 개인화된 추천 기능이나 스마트 홈 관리 등이 가능합니다. 

### 4. 가전 제품 혁신
- **비스포크 AI 가전**: 2025년형 비스포크 AI 제품군은 하이브리드 냉각 기능과 확장된 화면 옵션, 향상된 AI 음식 관리 시스템을 도입하여 일상생활을 보다 효율적으로 변화시켰습니다. 

이러한 변화들은 사용자의 요구를 충족시키고, 더 나은 사용 경험을 제공하기 위해 설계되었습니다. 더 자세한 정보는 아래의 링크에서 확인하실 수 있습니다:

- [Galaxy Unpacked 2025 총 정리 요약](https://r1.community.samsung.com/t5/%EA%B0%A4%EB%9F%AC%EC%8B%9C-s/%ED%9B%84%EA%B8%B0-galaxyunpacked-2025-%EC%B4%9D-%EC%A0%95%EB%A6%AC-%EC%9A%94%EC%95%BD-%EF%B8%8F/td-p/31413193)
- [2025 삼성 신제품 총정리!](https://blog.naver.com/PostView.naver?blogId=naramharam&logNo=223810258895)
- [AI 홈으로 일상을 혁신하다](https://news.samsung.com/kr/ai-%ED%99%88%EC%9C%BC%EB%A1%9C-%EC%9D%BC%EC%83%81%EC%9D%84-%ED%98%81%EC%8B%A0%ED%95%98%EB%8B%A4-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-2025%EB%85%84%ED%98%95-%EB%B9%84%EC%8A%A4)

이러한 혁신들은 사용자에게 더 나은 삶을 제공하기 위한 삼성전자의 끊임없는 노력의 일환입니다. 😊

elapsed time: 15.96 seconds
